In [36]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import sys
import json



#New imports
from textblob import TextBlob
import pandas as pd
import numpy as np
from textblob import TextBlob
import json

# Configuration

In [37]:
TWEET_COUNT = 2

## Tokens for the Tweepy API

In [65]:
ckey="hpbARu5lJlozST82JEdXYcaS7"
csecret="JpfbXATmdNPz6yIhFwmVU1mGU28QTICCuYkfqGibb25HV4x4VK"
atoken="1094363214043889665-UohVaHgV89P7OtSgUlWq6QJ6q30Yu1"
asecret="mz0RhOmUHN2ISAFcE3j0bOwZVxoAy5x5Ccn8RAiqYrRRI"

# Tweet Retrieval

In [90]:
class tweetFetcher:
    
    
    def __init__(self, ckey, csecret, atoken, asecret):
        
        auth = OAuthHandler(ckey, csecret)
        auth.set_access_token(atoken, asecret)
        self.api = tweepy.API(auth)
        
        self.actor_list = []
    
    #Manage API limit
    def limit_handled(self, cursor):
        while True:
            try:
                yield cursor.next()
            except tweepy.RateLimitError:
                time.sleep(15*60)
            except StopIteration:
                break
    
    #Generate actor list
    def create_actor_list(self):
        with open('test_actors.txt') as actor_file:
            actor_list = []
            for line in actor_file:
                actor_list.append(line.strip())
        self.actor_list = actor_list
        
    # assumes actor_list has no duplicates
    def run_query(self):
    
        with open("data_file.json", "w") as write_file:
            for actor in self.actor_list:
                tweet_list = []
                cursor = tweepy.Cursor(self.api.search, q=actor, lang='en', tweet_mode='extended').items(TWEET_COUNT)
                
        
                for tweet in self.limit_handled(cursor):
                    if hasattr(tweet, 'text'):
                        contents = tweet.text
                    else:
                        contents = tweet.full_text
                    datetime = tweet.created_at
                    tweet_tuple = datetime, contents
                    tweet_list.append(tweet_tuple)
                    json.dump(tweet_tuple, write_file, indent=4, sort_keys=True, default=str)



In [91]:
def main():

    tweet_fetcher = tweetFetcher(ckey, csecret, atoken, asecret)
    
    tweet_fetcher.create_actor_list()
    
    tweet_fetcher.run_query()


In [92]:
main()